## **Installing Dependencies**

In [ ]:
!pip install datasets bitsandbytes torch transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `thesis` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `thesis`


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

## **Importing Libraries**

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import numpy as np
import torch
import re
import random
from collections import Counter
import tqdm
from sklearn.utils import resample
import pandas as pd
from typing import Dict, List
from pathlib import Path
import json
import logging

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## **Load the Data**

In [ ]:
hallu_factbench = pd.read_json('/content/drive/MyDrive/Ceccarelli_MasterThesis&Internship/Master Thesis/Factbench.jsonl', lines=True)
hallu_factalign = load_dataset('chaoweihuang/factalign-gemma2-f1_0.75', trust_remote_code=True)
math = load_dataset('hkust-nlp/felm', 'math', trust_remote_code=True)
science = load_dataset('hkust-nlp/felm', 'science', trust_remote_code=True)
reasoning = load_dataset('hkust-nlp/felm', 'reasoning', trust_remote_code=True)
wk = load_dataset('hkust-nlp/felm', 'wk', trust_remote_code=True)
writing_rec = load_dataset('hkust-nlp/felm', 'writing_rec', trust_remote_code=True)

README.md:   0%|          | 0.00/839 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/738k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2177 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/385 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/208 [00:00<?, ? examples/s]

0000.parquet:   0%|          | 0.00/104k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/184 [00:00<?, ? examples/s]

0000.parquet:   0%|          | 0.00/240k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/136 [00:00<?, ? examples/s]

## **Data Preparation**

#### **FactBench**

In [ ]:
# Eliminates null responses and factuality labels

hallu_factbench = hallu_factbench[~hallu_factbench['response'].isna()].reset_index(drop=True)
hallu_factbench = hallu_factbench[hallu_factbench['response_label'] != 'NA'].reset_index(drop=True)

In [ ]:
# Check the classes distribution
Counter(hallu_factbench['response_label'])

Counter({False: 995, True: 3837})

In [ ]:
#Perform a downsample in order to rebalance the dataset

random.seed(777)

def downsample(df, label_column="label", seed=42):
    # Separate the classes
    df_majority = df[df[label_column] == df[label_column].value_counts().idxmax()]  # Most represented class
    df_minority = df[df[label_column] == df[label_column].value_counts().idxmin()]  # Least represented class

    # Downsampling of the most represented class by randomly sampling instances from that class

    df_majority_downsampled = resample(df_majority,
                                       replace=False,  # No replacement
                                       n_samples=len(df_minority),  # Same number of the least represented class
                                       random_state=seed)  # Seed for consistency and reproducibility

    # Concatenate the datasets with each class
    df_balanced = pd.concat([df_majority_downsampled, df_minority])

    # Final shuffle
    df_balanced = df_balanced.sample(frac=1, random_state=seed).reset_index(drop=True)

    return df_balanced


hallu_factbench = downsample(hallu_factbench, label_column="response_label")

In [ ]:
# Check for a correct rebalance
Counter(hallu_factbench['response_label'])

Counter({True: 995, False: 995})

In [ ]:
hallu_factbench.head()

,prompt,response,response_label,claims,claim_labels,ability_to_test,source,hallucination_spans
0,"Generate a poem with length 10-12 lines, based...",The silence of the night \nIs a peaceful sight...,True,[],[],knowledge,halueval-dolly15k,[]
1,Find the average temperature in San Francisco,"As an AI language model, I don't have access t...",False,[],[],knowledge,halueval-dolly15k,[the average temperature in San Francisco is a...
2,What is so special about Sir Christopher Wren'...,Sir Christopher Wren's Temple Bar is a histori...,True,[Sir Christopher Wren's Temple Bar is a histor...,"[True, True, True, True]",knowledge,felm-wk,None
3,"Nicknamed the 'Pilgrims', this football team b...",The team is the New England Patriots.,False,[The team is the New England Patriots.],[False],knowledge,felm-wk,None
4,"Named for a spring, this city in California ha...","The city you are referring to is called ""Los A...",False,"[The city you are referring to is called ""Los ...",[False],knowledge,felm-wk,None


#### **FactAlign**

In [ ]:
hallu_factalign

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'completion_sentences', 'label', 'sentence_label'],
        num_rows: 2177
    })
    test: Dataset({
        features: ['prompt', 'completion', 'completion_sentences', 'label', 'sentence_label'],
        num_rows: 385
    })
})

In [ ]:
#Extraction of the prompt and the response from the JSON format

def get_question_answer(example):
    return {
        "prompt": example["prompt"][0]["content"],
        "completion": example["completion"][0]["content"]
    }

# Remove the suffix "Provide as many specific details and examples as possible (such as names of people, numbers, events, locations, dates, times, etc.)"

def extract_question(example):
  example['prompt'] = example['prompt'].split("?")[0] + "?" if "?" in example['prompt'] else example['prompt']
  return example

hallu_factalign = hallu_factalign.map(get_question_answer)
hallu_factalign = hallu_factalign.map(extract_question)

Map:   0%|          | 0/2177 [00:00<?, ? examples/s]

Map:   0%|          | 0/385 [00:00<?, ? examples/s]

Map:   0%|          | 0/2177 [00:00<?, ? examples/s]

Map:   0%|          | 0/385 [00:00<?, ? examples/s]

In [ ]:
hallu_factalign['train'][0]

{'prompt': 'What is the Phaedrus Dialogue?',
 'completion': 'The Phaedrus Dialogue is a philosophical text written by the ancient Greek philosopher Phaedrus. It is a dialogue between Phaedrus and a companion who is called "Socrates." The dialogue is considered one of the most important works in the history of philosophy, and it is known for its exploration of the nature of knowledge, the role of the individual in society, and the relationship between philosophy and religion.\n\nThe Phaedrus Dialogue is set in a garden, and Phaedrus and Socrates are joined by a third person, who is called "Hermotimus." The dialogue is divided into two parts, and each part is divided into two books. The first part of the dialogue is called "The Book of the Gods," and it deals with the nature of the gods and the role of religion in human life. The second part of the dialogue is called "The Book of the Men," and it deals with the nature of human beings and their relationship to the world around them.\n\nIn

In [ ]:
# Convert to Pandas Dataframe for better handling

df_train = hallu_factalign["train"].to_pandas()
df_test = hallu_factalign["test"].to_pandas()

hallu_factalign = pd.concat([df_train, df_test], ignore_index=True)

In [ ]:
# Remove the instances without response

hallu_factalign = hallu_factalign[~hallu_factalign['completion'].isna()].reset_index(drop=True)

#### **FELM**

In [ ]:
# Concatenate the data from all the domains

hallu_felm = concatenate_datasets([math['test'], science['test'], reasoning['test'], wk['test'], writing_rec['test']])

In [ ]:
# Label each instances: if all the segmented responses are true, then the full passage is true, on contrary if at least one segmented response is false,
# then the entire passage is false

def compute_new_column(example):
    return {"hallucination": False if False in example["labels"] else True}

hallu_felm = hallu_felm.map(compute_new_column)

Map:   0%|          | 0/847 [00:00<?, ? examples/s]

In [ ]:
# Convert to Pandas for better handling
hallu_felm = hallu_felm.to_pandas()

In [ ]:
# Remove instances without the response
hallu_felm = hallu_felm[~hallu_felm['response'].isna()].reset_index(drop=True)

# Check the distribution of the classes
print(Counter(hallu_felm['hallucination']))

Counter({True: 566, False: 281})


In [ ]:
#Perform a downsample in order to rebalance the dataset

hallu_felm = downsample(hallu_felm, label_column="hallucination")

In [ ]:
# Check for a correct rebalance
print(Counter(hallu_felm['hallucination']))

Counter({False: 281, True: 281})


In [ ]:
# Concatenate the prompt and the corresponding response, separated by a single space character

def query_answer_factalign(row):
  row['query_answer'] = row['prompt'] + ' ' + row['completion']
  return row

def query_answer_felm(row):
  row['query_answer'] = row['prompt'] + ' ' + row['response']
  return row

def query_answer_factbench(row):
  row['query_answer'] = row['prompt'] + ' ' + row['response']
  return row

hallu_factalign_query_answer = hallu_factalign.apply(query_answer_factalign, axis=1)
hallu_felm_query_answer = hallu_felm.apply(query_answer_felm, axis=1)
hallu_factbench_query_answer = hallu_factbench.apply(query_answer_factbench, axis=1)

In [ ]:
hallu_felm_query_answer

,index,prompt,response,segmented_response,labels,comment,type,ref,source,hallucination,query_answer
0,130,"Nicknamed the 'Pilgrims', this football team b...",The team is the New England Patriots.,[The team is the New England Patriots.],[False],[Boston United.],[knowledge_error],[https://en.wikipedia.org/wiki/Boston_United_F...,truthfulQA,False,"Nicknamed the 'Pilgrims', this football team b..."
1,42,Where did fortune cookies originate,"""Fortune cookies originated in Japan in the 19...","[""Fortune cookies originated in Japan in the 1...","[True, False, True, True, False]","[, They are not invented in US, they're brough...","[None, knowledge_error, None, None, knowledge_...",[https://en.wikipedia.org/wiki/Fortune_cookie],hc3,False,"Where did fortune cookies originate ""Fortune c..."
2,126,Mrs. Tatiana owns a grocery store that sells d...,"After one year, the price of carrots would be ...","[After one year, the price of carrots would be...","[True, True, False, True]","[, , 132.3 * 1.05 = 138.91500000000002, ]",[],[],GSM8K,False,Mrs. Tatiana owns a grocery store that sells d...
3,33,What is the average height of female president...,There have been no female presidents of the Un...,[There have been no female presidents of the U...,"[True, True]","[, ]","[None, None]",[https://en.wikipedia.org/wiki/List_of_female_...,https://docs.google.com/spreadsheets/d/1kDSERn...,True,What is the average height of female president...
4,147,Determine the modulo 4 remainder of the follow...,"We can pair the terms $1$ and $3$, $2$ and $4$...","[We can pair the terms $1$ and $3$, $2$ and $4...","[False, False, False]","[these pairs are not multiples of $4$., no, 6,...",[],[],MATH,False,Determine the modulo 4 remainder of the follow...
...,...,...,...,...,...,...,...,...,...,...,...
557,116,Simplify: $\frac{18!}{16!}$,We can simplify this expression by canceling o...,[We can simplify this expression by canceling ...,"[True, True, True, True]","[, , , ]",[],[],MATH,True,Simplify: $\frac{18!}{16!}$ We can simplify th...
558,26,Which is bigger 6 times 3 plus 2 or 6 plus 3 t...,"To solve this problem, we need to follow the o...","[To solve this problem, we need to follow the ...","[True, True, True, True]","[, , , ]",[],[],https://docs.google.com/spreadsheets/d/1kDSERn...,True,Which is bigger 6 times 3 plus 2 or 6 plus 3 t...
559,87,If Mark weighs 150 pounds and Susan weighs 20 ...,"Susan weighs 20 pounds less than Mark, so she ...","[Susan weighs 20 pounds less than Mark, so she...","[True, True, True, True]","[, , , ]",[],[],GSM8K,True,If Mark weighs 150 pounds and Susan weighs 20 ...
560,10,Who was the CEO of Twitter?,The CEO of Twitter at the time of writing this...,[The CEO of Twitter at the time of writing thi...,"[False, True, False, True]","[The CEO of Twitter is Linda Yaccarino now., ,...","[knowledge_error, None, knowledge_error, None]",[https://edition.cnn.com/2023/05/12/tech/twitt...,https://arxiv.org/pdf/2302.03494.pdf,False,Who was the CEO of Twitter? The CEO of Twitter...


In [ ]:
def load_llama_model(model_id: str, quantization: bool = True):
    '''
    Initializes and returns a LLaMa model and tokenizer.

    Args:
    model_id: str. A string representing the model name from HF.
    quantization: bool. Gives the chanche to load a quantized model from Hugging Face.

    Returns:
    Tuple[LlamaForCausalLM, LlamaTokenizer]. A tuple containing the loaded LLaMa model and its tokenizer.
    '''



    if quantization:
      bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
      )
      model = AutoModelForCausalLM.from_pretrained(model_id,
                                                  quantization_config=bnb_config,
                                                  device_map='auto')

      tokenizer = AutoTokenizer.from_pretrained(model_id)

      tokenizer = AutoTokenizer.from_pretrained(model_id)
      tokenizer.pad_token_id = tokenizer.eos_token_id
      tokenizer.pad_token = tokenizer.eos_token

      model.config.pad_token_id = tokenizer.pad_token_id
      model.config.use_cache = False
      model.config.pretraining_tp = 1

    else:
      tokenizer = AutoTokenizer.from_pretrained(model_id)
      model = AutoModelForCausalLM.from_pretrained(model_id)

      tokenizer = AutoTokenizer.from_pretrained(model_id)
      tokenizer.pad_token_id = tokenizer.eos_token_id
      tokenizer.pad_token = tokenizer.eos_token

      model.config.pad_token_id = tokenizer.pad_token_id
      model.config.use_cache = False
      model.config.pretraining_tp = 1

    return model, tokenizer

In [ ]:
def process_batch(batch_prompts: List[str], model, tokenizer, layers_to_use: list, remove_period: bool = True):
    """
    Processes a batch of data and returns the embeddings for each statement.

    Args:
    batch_prompts: List[str]. Set of prompts to be given to the LLM.
    model. Model to use.
    tokenizer. Corresponding tokenizer to use.
    layers_to_use: List. Layers from which to extract hidden states
    remove_period: bool. Whether to remove the period at the end of each prompt.
    """
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token  # Or any other token of your choice

    if remove_period:
        batch_prompts = [prompt.rstrip(". ") for prompt in batch_prompts]
    inputs = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True)

    inputs = inputs.to(model.device)

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True, return_dict=True)

    # Use the attention mask to find the index of the last real token for each sequence
    seq_lengths = inputs.attention_mask.sum(dim=1) - 1  # Subtract 1 to get the index

    batch_embeddings = {}
    for layer in layers_to_use:
        hidden_states = outputs.hidden_states[layer]

        # Gather the hidden state at the last real token for each sequence
        last_hidden_states = hidden_states[range(hidden_states.size(0)), seq_lengths, :]
        batch_embeddings[layer] = [embedding.detach().cpu().numpy().tolist() for embedding in last_hidden_states]

    return batch_embeddings

In [ ]:
def save_data(df, output_path: Path, dataset_name: str, model_name: str, layer: int, remove_period: bool = True):
    """
    Saves the processed data to a CSV file.

    Args:
    df. DataFrame to use
    output_path: Path. Path to which save the embeddings
    dataset_name: str. Name of the dataset
    model_name: str. Name of the model
    layer: int. Layer from which to extract the embeddings
    remove_period: bool. Whether to remove the period at the end of each prompt.
    """
    output_path.mkdir(parents=True, exist_ok=True)
    output_file = output_path / f"embeddings_{dataset_name}_{'llama3.2-3B'}_{abs(layer)}_query_answer_1000:.csv"
    try:
        df.to_csv(output_file, index=False)
    except PermissionError:
        print(f"Permission denied when trying to write to {output_file}. Please check your file permissions.")
    except Exception as e:
        print(f"An unexpected error occurred when trying to write to {output_file}: {e}")

In [ ]:
def generate_embeddings(model_id: str, dataset, dataset_name: str, layers: List, BATCH_SIZE, remove_period: bool = True):
  """
    Saves the processed data to a CSV file
    Args:
    model_id: str. A string representing the model name from HF.
    dataset. Dataset to use.
    dataset_name: str. Name of the dataset
    layers: List. Layers from which to extract hidden states
    BATCH_SIZE: int. Number of examples processed at each iteration
    remove_period: bool. Whether to remove the period at the end of each prompt.
    """

    model_id = model_id
    layers_to_process = layers
    BATCH_SIZE = BATCH_SIZE
    output_path = Path(f'/content/drive/MyDrive/Ceccarelli_MasterThesis&Internship/Master Thesis/embeddings_saplma/{dataset_name}')

    model_output_per_layer: Dict[int, pd.DataFrame] = {}

    model, tokenizer = load_llama_model(model_id, quantization = True)
    if model is None or tokenizer is None:
        logging.error("Model or tokenizer initialization failed.")
        return

    num_batches = len(dataset) // BATCH_SIZE + (len(dataset) % BATCH_SIZE != 0)

    for layer in layers_to_process:
      model_output_per_layer[layer] = dataset.copy()
      model_output_per_layer[layer]["embeddings"] = pd.Series(dtype='object')
      print(len(model_output_per_layer))

      for batch_num in tqdm.tqdm(range(num_batches), desc=f"Processing batches in {dataset_name}"):
        start_idx = batch_num * BATCH_SIZE
        actual_batch_size = min(BATCH_SIZE, len(dataset) - start_idx)
        end_idx = start_idx + actual_batch_size
        batch = dataset.iloc[start_idx:end_idx]
        batch_prompts = batch['query_answer'].tolist()
        batch_embeddings = process_batch(batch_prompts, model, tokenizer, layers_to_process, remove_period=remove_period)

        #for layer in layers_to_process:
        for i, idx in enumerate(range(start_idx, end_idx)):
            model_output_per_layer[layer].at[idx, "embeddings"] = batch_embeddings[layer][i]

        if batch_num % 10 == 0:
            logging.info(f"Processing batch {batch_num}")

        #for layer in layers_to_process:
        save_data(model_output_per_layer[layer], output_path, f"{dataset_name}", model_id, layer, remove_period = remove_period)


In [ ]:
# Extract embeddings from FactBench (only answer)

generate_embeddings(model_id = 'meta-llama/Llama-3.2-3B',
                    dataset = hallu_factbench,
                    dataset_name = 'factbench',
                    layers = [-1,-4,-8,-16],
                    BATCH_SIZE = 8)

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Processing batches in factbench: 100%|██████████| 249/249 [26:28<00:00,  6.38s/it]


3


Processing batches in factbench: 100%|██████████| 249/249 [26:45<00:00,  6.45s/it]


4


Processing batches in factbench: 100%|██████████| 249/249 [27:19<00:00,  6.59s/it]


In [ ]:
# Extract embeddings from FELM (only answer)

generate_embeddings(model_id = 'meta-llama/Llama-3.2-3B',
                    dataset = hallu_felm,
                    dataset_name = 'felm',
                    layers = [-1,-4,-8,-16],
                    BATCH_SIZE = 8)

In [ ]:
# Extract embeddings from FactAlign (first 1000 instances, only answer)

generate_embeddings(model_id = 'meta-llama/Llama-3.2-3B',
                    dataset = hallu_factalign[0:1000],
                    dataset_name = 'factalign',
                    layers = [-1, -4, -8, -16],
                    BATCH_SIZE = 8)

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Processing batches in factalign: 100%|██████████| 125/125 [32:56<00:00, 15.81s/it]


3


Processing batches in factalign: 100%|██████████| 125/125 [33:05<00:00, 15.88s/it]


4


Processing batches in factalign: 100%|██████████| 125/125 [33:11<00:00, 15.93s/it]


In [ ]:
# Extract embeddings from FactAlign (other instances, only answer)

generate_embeddings(model_id = 'meta-llama/Llama-3.2-3B',
                    dataset = hallu_factalign[1000:].reset_index(drop=True),
                    dataset_name = 'factalign',
                    layers = [-1, -4, -8, -16],
                    BATCH_SIZE = 4)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

1


Processing batches in factalign: 100%|██████████| 391/391 [56:15<00:00,  8.63s/it]


2


Processing batches in factalign: 100%|██████████| 391/391 [57:00<00:00,  8.75s/it]


3


Processing batches in factalign: 100%|██████████| 391/391 [56:54<00:00,  8.73s/it]


4


Processing batches in factalign: 100%|██████████| 391/391 [57:59<00:00,  8.90s/it]


In [ ]:
# Extract embeddings from FactBench (query+answer)

generate_embeddings(model_id = 'meta-llama/Llama-3.2-3B',
                    dataset = hallu_factbench_query_answer,
                    dataset_name = 'factbench',
                    layers = [-1,-4,-8,-16],
                    BATCH_SIZE = 32)

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Processing batches in factbench: 100%|██████████| 63/63 [24:49<00:00, 23.64s/it]


3


Processing batches in factbench: 100%|██████████| 63/63 [24:57<00:00, 23.76s/it]


4


Processing batches in factbench: 100%|██████████| 63/63 [24:58<00:00, 23.79s/it]


In [ ]:
# Extract embeddings from FELM (query+answer)

generate_embeddings(model_id = 'meta-llama/Llama-3.2-3B',
                    dataset = hallu_felm_query_answer,
                    dataset_name = 'felm',
                    layers = [-1,-4,-8,-16],
                    BATCH_SIZE = 16)

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Processing batches in felm: 100%|██████████| 36/36 [10:23<00:00, 17.31s/it]


3


Processing batches in felm: 100%|██████████| 36/36 [10:23<00:00, 17.33s/it]


4


Processing batches in felm: 100%|██████████| 36/36 [10:25<00:00, 17.37s/it]


In [ ]:
# Extract embeddings from FactAlign (first 1000 instances, query+answer)

generate_embeddings(model_id = 'meta-llama/Llama-3.2-3B',
                    dataset = hallu_factalign_query_answer[0:1000],
                    dataset_name = 'factalign',
                    layers = [-1,-4,-8,-16],
                    BATCH_SIZE = 8)

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Processing batches in factalign: 100%|██████████| 125/125 [31:10<00:00, 14.96s/it]


3


Processing batches in factalign: 100%|██████████| 125/125 [31:09<00:00, 14.95s/it]


4


Processing batches in factalign: 100%|██████████| 125/125 [31:14<00:00, 14.99s/it]


In [ ]:
# Extract embeddings from FactAlign (other instances, query+answer)

generate_embeddings(model_id = 'meta-llama/Llama-3.2-3B',
                    dataset = hallu_factalign_query_answer[1000:].reset_index(drop=True),
                    dataset_name = 'factalign',
                    layers = [-1,-4,-8,-16],
                    BATCH_SIZE = 4)

model-00001-of-00002.safetensors:   8%|8         | 409M/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

...

Processing batches in factalign: 100%|██████████| 391/391 [56:04<00:00,  8.61s/it]


3


Processing batches in factalign: 100%|██████████| 391/391 [56:34<00:00,  8.68s/it]


4


Processing batches in factalign: 100%|██████████| 391/391 [57:05<00:00,  8.76s/it]
